# Example: Two-dimensional beam biting its root

In [9]:
from beef import fe
import numpy as np

import pyvista as pv
pv.set_jupyter_backend('trame')

First, the `fe.Section` class is used to construct a Section object describing the properties of the beam:

In [10]:
sections = [fe.Section(E=100, A=100, I_y=1, poisson=0.3)] 	# list of Section objects

Thereafter, the beam is discretized into 20 elements and is given a length of 10 m. A node matrix and an element matrix is constructed (typical input to commercial FE software), to construct the part. The beam is constrained in all degrees of freedom in node 1, by creating and inputting a list of a single `fe.Constraint` object.

In [13]:
# Define mesh
els = 20
L = 10
node_labels = np.arange(1, els+2)
x = (node_labels - 1)/els*L
node_matrix = np.vstack([node_labels.T, x.T, x.T*0]).T 	# rows: label, x, y
element_matrix = np.vstack([np.arange(1,els+1), node_labels[0:-1], node_labels[1:]]).T # rows: label, n1, n2

# Define constraints
constraints = [fe.Constraint([1], dofs='all', node_type='beam2d')] 	# list of Constraint objects

# Define part and plot part
part = fe.Part(node_matrix, element_matrix, sections=sections, constraints=constraints)   # Part(ElDef) object
part.plot(plot_nodes=True, node_labels=True)

Widget(value="<iframe src='http://localhost:57262/index.html?ui=P_0x234dbfb0250_5&reconnect=auto' style='width…

Furthermore, a moment $M=2\pi E I/L$ (analytical moment needed for beam to bite its tail) is applied along degree-of-freedom index 2 at the rightmost node (last label in `node_labels`):

In [14]:
# Define force
M = [2*np.pi*sections[0].E*sections[0].I[0]/L]	# analytical moment needed for beam to bite its tail
dof_ix = 2	# i.e., bending moment
force_nodelabels = [node_labels[-1]]	# apply force on last node (rightmost)
forces = [fe.Force(force_nodelabels, dof_ix, M, t=1)] 	# list of Force objects

Finally, the analysis object is initiated by the `Part` object, the list of `Force` objects, and parameters `dt` (time step) and `tol` (dictionary describing tolerance criteria for displacements, 'u' and residual forces, 'r'). The analysis is furthermore run by invoking `analysis.run_static()`:

In [15]:
# Define and run analysis
dt = 0.1 	# time step
tol = dict(r=0.1) 	#tolerances, specified residual max. 0.1
analysis = fe.Analysis(part, forces=forces, dt=dt, tol=tol)  # Analysis object, tmax not specified => tmax=1.0
analysis.run_static() # run static analysis

Static analysis:   0%|          | 0/11 [00:00<?, ?it/s]

The resulting deformation is plotted as follows:

In [16]:
analysis.eldef.plot(node_labels=False, element_labels=False, plot_nodes=True, plot_states=['undeformed', 'deformed'])

Widget(value="<iframe src='http://localhost:57262/index.html?ui=P_0x234e4eacdf0_6&reconnect=auto' style='width…